In [47]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', 500)

public prompt

In [ ]:
import pandas as pd

df1 = pd.read_csv("./data/0401/nbroad-v2.csv")
df2 = pd.read_csv("./data/0401/gemma_suppl_rewrite.csv")

df2['rewrite_prompt'] = df2['rewrite_prompts']

df = pd.concat([df1[['rewrite_prompt']], df2[['rewrite_prompt']]])

# df = pd.read_csv("./data/0304/seed_final.csv")

df = df[df['rewrite_prompt'].str.split(" ").apply(len) <= 15]
df = df[~df['rewrite_prompt'].str.contains(r"if|preserving|maintaining|/|\(|\.|\n", regex=True)]
df = df[~df['rewrite_prompt'].str.endswith("-")]
df = df[~df['rewrite_prompt'].str.contains(f"one act|Christianity|back-alley city|first line as|no exceptions|verbs except|with a twist|every moment is|story titled|watched three episodes|being replaced with|make it clear that|character-customization-menu|or another|completely unknown to|but|assuming|various popular franchises", regex=True)]
df['rewrite_prompt'] = df['rewrite_prompt'].str.replace(r"\.$", "", regex=True)
df['rewrite_prompt'] = df['rewrite_prompt'].apply(lambda x: x.strip())

print(df.shape)
prompts = df['rewrite_prompt'].tolist()

df.iloc[:12]['rewrite_prompt']

dung's prompt

In [11]:
import pandas as pd
import random

df_prompts = pd.read_csv("./data/0408/data/diverse_prompts.csv")

df_prompts['type'].value_counts()
# Get the count of each type
type_counts = df_prompts['type'].value_counts()

# Find the minimum count among the types
max_count = max(type_counts)

# Create an empty list to store the balanced DataFrame
balanced_df = []

# Duplicate random rows to balance the count
for type_name, count in type_counts.items():
    # Get the rows of the current type
    type_rows = df_prompts[df_prompts['type'] == type_name]
    
    # Duplicate random rows from the current type to match the minimum count
    duplicated_rows = type_rows.sample(n=max_count - count, replace=True)
    
    # Append the original and duplicated rows to the balanced DataFrame
    balanced_df.append(type_rows)
    balanced_df.append(duplicated_rows)

# Concatenate the balanced DataFrame
balanced_df = pd.concat(balanced_df)

# Shuffle the rows of the balanced DataFrame
balanced_df = balanced_df.sample(frac=1).reset_index(drop=True)

# Print the count of each type in the balanced DataFrame
print(balanced_df['type'].value_counts())
print(balanced_df.shape)

df_prompts = balanced_df


type
specific    135
literary    135
general     135
modern      135
Name: count, dtype: int64
(540, 2)


In [ ]:
# encode
from sentence_transformers import SentenceTransformer

t5_base = SentenceTransformer('sentence-t5-base')

embeddings = t5_base.encode(df['rewrite_prompt'].tolist()).tolist()

embeddings

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

def diverse_subset(vectors, k):
    # Convert vectors to numpy array
    X = np.array(vectors)
    
    # Perform K-means clustering
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X)
    
    # Get the cluster labels
    cluster_labels = kmeans.labels_
    
    # Find the centroids of each cluster
    centroids = kmeans.cluster_centers_
    
    # Initialize a list to store the diverse subset
    diverse_subset = []
    
    # Select one vector from each cluster to include in the diverse subset
    for i in range(k):
        cluster_indices = np.where(cluster_labels == i)[0]
        if len(cluster_indices) > 0:
            # Choose a representative vector from the cluster (e.g., centroid)
            representative_index = np.argmin(np.linalg.norm(X[cluster_indices] - centroids[i], axis=1))
            diverse_subset.append(cluster_indices[representative_index])
    
    return diverse_subset

subset = diverse_subset(embeddings, 4)
print("Diverse subset:", len(subset))

In [ ]:
df_shuffle = df.iloc[subset].sample(frac=1).reset_index(drop=True)
df_shuffle.to_csv("./data/0401/nbroad-v2-gemma_suppl_rewrite.csv", index=False, columns=['rewrite_prompt'])

df_shuffle.iloc[0]['rewrite_prompt']

# Build predict datasets

In [17]:
import random

random.seed(42)

# df_prompts = pd.read_csv("./data/0401/nbroad-v2-gemma_suppl_rewrite.csv")
df_text = pd.read_csv("./data/0401/150_suppl_original_text.csv")
df_text = df_text[df_text['original_text'].str.split(" ").apply(len) <= 100]
df_text = df_text[~df_text['original_text'].str.contains(r"\[\w+\]", regex=True)]

# prompts = df_prompts.iloc[:200]['rewrite_prompt'].tolist()

df_rows = []
for i, row in df_prompts.iterrows():
    texts = df_text.iloc[random.choices(range(len(df_text)), k=2)]['original_text'].to_list()
    for t in texts:
        df_rows.append({'rewrite_prompt': row['prompt'], 'original_text': t, 'type': row['type']})

df = pd.DataFrame(df_rows)
df = df.sample(frac=1).reset_index(drop=True)

df.to_csv("./data/0407/diverse_dataset.csv", index=False)

df.shape

(1080, 3)